In [1]:
import sys
print(sys.version)
import os

3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]


In [2]:
# !pip install pymupdf chromadb
# !pip install -U langchain-chroma
# !pip install langchain-ollama
# !pip install -qU langchain-pymupdf4llm

In [3]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader
loader = PyMuPDF4LLMLoader("Sivamani_Resume.pdf")
docs = loader.load()

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [4]:
print(docs[0].page_content[0:50])

# **SIVAMANI R**

[(+91) 875-427-3235 | sivamanira


In [5]:
import pprint
pprint.pp(docs[0].metadata)

{'producer': 'Microsoft® Word 2013',
 'creator': 'Microsoft® Word 2013',
 'creationdate': '2025-11-24T12:44:14+05:30',
 'source': 'Sivamani_Resume.pdf',
 'file_path': 'Sivamani_Resume.pdf',
 'total_pages': 1,
 'format': 'PDF 1.5',
 'title': '',
 'author': 'Sivamani R',
 'subject': '',
 'keywords': '',
 'moddate': '2025-11-24T12:44:14+05:30',
 'trapped': '',
 'modDate': "D:20251124124414+05'30'",
 'creationDate': "D:20251124124414+05'30'",
 'page': 0}


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # small enough for accuracy
    chunk_overlap=50,     # keeps context between chunks
)
chunks = text_splitter.split_documents(docs)


In [9]:
len(chunks)

9

In [10]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)
db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./resume_chroma"
)
print('successful')

successful


**access from db**

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

db = Chroma(
    persist_directory="./resume_chroma",
    embedding_function=embeddings
)


C:\Users\Arul prakasam\AppData\Local\Temp\ipykernel_13452\3886947901.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


In [14]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)


In [17]:
query = "What are my skills?"

docs = retriever.invoke(query)



In [19]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3",
    temperature=0
)


In [21]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
You are a resume assistant.

Answer ONLY using the context below.
Do NOT add information.
If the answer is not present, say "Not found in resume".

Context:
{context}

Question:
{question}
""")


In [22]:
question = "What are my skills?"

docs = retriever.invoke(question)

context = "\n".join([doc.page_content for doc in docs])

final_prompt = prompt.format(
    context=context,
    question=question
)

answer = llm.invoke(final_prompt)
print(answer)


Your skills include:

* Machine learning
* Deep learning
* Natural language processing
* Regression
* Classification
* Clustering
* CNN models
* Scikit-learn
* TensorFlow
* Generative AI
* Large language models
* Python
* NumPy
* SciPy
* Pandas
* Matplotlib
* Flask
* FastAPI
